In [ ]:
%pip install google-genai
%pip install matplotlib
%pip install os
%pip install dotenv

In [40]:
from google import genai
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from dotenv import load_dotenv
from google.genai import types
from io import BytesIO
from PIL import Image
from IPython.display import Markdown

In [ ]:
def create_genai_client():
    import os

    # api_key = os.getenv("GOOGLE_API_KEY")
    # if not api_key:
    #     raise ValueError("GOOGLE_API_KEY not found in .env file")
    
    cloud_api_key = os.getenv("GOOGLE_CLOUD_API_KEY")
    if not cloud_api_key:
        raise ValueError("GOOGLE_CLOUD_API_KEY not found in .env file")
    
    # Configure the client with your API key
    # client = genai.Client(api_key=api_key, http_options={'api_version': 'v1alpha'})
    client = genai.Client(
        vertexai=True, 
        api_key=cloud_api_key, 
    )

    return client

In [ ]:
load_dotenv()

# Configure the client with your API key
client = create_genai_client()

tools = [types.Tool(google_search=types.GoogleSearch())]

In [ ]:
def print_token_usage(response: types.GenerateContentResponse):
    if response and response.usage_metadata:
        usage_metadata = response.usage_metadata
        input_token_count = usage_metadata.prompt_token_count
        output_token_count = usage_metadata.candidates_token_count
        total_token_count = usage_metadata.total_token_count
        thought_token_count = usage_metadata.thoughts_token_count
        cached_token_count = usage_metadata.cached_content_token_count
        print(f"Input: {input_token_count}, Output: {output_token_count}, Thought: {thought_token_count}, Cached: {cached_token_count} Total: {total_token_count}")
        
def get_num_citations(grounding_supports: list[types.GroundingSupport] | None):
    num_citations = 0
    for support in grounding_supports or []:
        if support.grounding_chunk_indices:
            num_citations = num_citations + len(support.grounding_chunk_indices)

    return num_citations

def build_citations(grounding_supports: list[types.GroundingSupport] | None, grounding_chunks: list[types.GroundingChunk] | None):
    citations: list[str] = []
    for support in grounding_supports:
        if support.grounding_chunk_indices:
            for i in support.grounding_chunk_indices:
                uri = ''
                if grounding_chunks and i < len(grounding_chunks):
                    chunk = grounding_chunks[i]
                    uri = chunk.web.uri if chunk.web and chunk.web.uri else ''
                if uri:
                    citations.append(uri)

    text_citations = "\nCitation link: ".join(citations)
    return "Citation link: " + text_citations

def execute_prompt(prompt: str):
    response = client.models.generate_content(
        model="gemini-3-pro-preview",
        contents=[types.Content(
            role="user",
            parts=[types.Part(text=prompt)]
        )],
        config=types.GenerateContentConfig(
            tools=tools,
            thinking_config=types.ThinkingConfig(
                thinking_level=types.ThinkingLevel.HIGH
            ),
        )
    )

    print_token_usage(response)
    return response

def print_response(response: types.GenerateContentResponse):
    # print the response
    display(Markdown(f"Response:\n {response.text}"))

    grounding_metadata = response.candidates[0].grounding_metadata if response.candidates and response.candidates[0] and response.candidates[0].grounding_metadata else None
    grounding_chunks = grounding_metadata.grounding_chunks if grounding_metadata and grounding_metadata.grounding_chunks else None
    grounding_supports = grounding_metadata.grounding_supports if grounding_metadata and grounding_metadata.grounding_supports else None
    if grounding_supports and grounding_chunks:
        print (f"{get_num_citations(grounding_supports)} Citations")
        citations = build_citations(grounding_supports=grounding_supports, grounding_chunks=grounding_chunks)
        print(citations)

    web_search_queries = grounding_metadata.web_search_queries if grounding_metadata and grounding_metadata.web_search_queries else None
    if web_search_queries and len(web_search_queries) > 0:
        for query in web_search_queries:
            if query:
                print (f"Query -> {query}")

In [ ]:
response = execute_prompt(prompt="""
    Please search for the latest confirmed weather forecast for Taiwan for the dates listed below in 2025. specifically looking at Taipei and Taoyuan.
    
    Present the results in a single clear table with the following columns: Date, Location, Weather (Condition, Temp in °C, Precipitation %), and Appropriate Clothing.

    The dates and locations are:
    1. Taipei: November 26, 27, 28, and 29.
    2. Taoyuan: November 30 and December 1.
""")

print_response(response)

In [ ]:
weather_forecast = response.text
response = execute_prompt(prompt=f"""
    Based on the weather forecast you just retrieved and the itinerary details below, please act as a professional travel planner and generate a comprehensive packing list.

    Trip Itinerary:
    - November 26, 2025 - November 29, 2025: Taipei (City exploration, night markets, extensive walking).
    - November 30, 2025 - December 1, 2025: Taoyuan (Transitioning closer to the airport/coastal area).

    Please categorize the packing list into:

    1. Clothing: Specific recommendations based on the temperature and rain forecast you found. Suggest layers suitable for transitioning between humid outdoor weather, likely rain, and strong indoor air conditioning. Recommend shoes suitable for wet pavement and high daily step counts.
    2. Documents: Entry requirements for Taiwan, digital backups, and essential travel apps.
    3. Financial: Advice on carrying cash (TWD) vs. credit cards, specifically distinguishing between Night Market needs vs. Malls/Department stores.
    4. Electronics: Adapter types (confirm if US plugs work in Taiwan), power banks, and connectivity (eSIM vs. physical SIM).
    5. Essentials: Toiletries, Feminine products and specific items for humid weather. 
    6. Taoyuan Logistics: Any specific tips for moving between Taipei and Taoyuan (e.g., keeping travel documents accessible).
    7. Airport Logistics: Any specific tips for moving between the hotel I am staying in Taoyuan and the airport.

    Special Request: Include a "Taiwan Essentials" section, including the EasyCard (transport card) and specific umbrella recommendations.
    
    Weather forecast:
    {weather_forecast}
    """)

print_response(response)

In [ ]:
response = execute_prompt(prompt="""
    Please search the web for my Cathay Pacific flight on November 26, 2025 from Hong Kong to Taipei. The flight number is CX530.
    The departure flight is CX495 on December 1, 2025. 
      
    Present the results in a single clear table with the following columns: Date, From, To, Flight Provider, Flight  Number, Plane Model, Departure, Arrival.
    The From column includes location, airport code, and terminal, in the format of  <Location> (<Code>) - <Terminal>
    Similarly, the To column includes the the same information in the same format."""
)

print_response(response)